# TP : Programmation dynamique

## Coefficient binomial

On souhaite calculer $\binom{n}{k}$ par programmation dynamique, en utilisant la formule de Pascal :

$$\binom{n}{k} = \binom{n - 1}{k - 1} + \binom{n - 1}{k}$$

````{admonition} Question
 Que peut-on prendre comme cas de base ?
````

````{admonition} Question
 Écrire une fonction récursive `binom_rec(n, k)` renvoyant $\binom{n}{k}$ à partir de la formule ci-dessus. Expliquer pourquoi la complexité de cette fonction est très mauvaise.
````

In [20]:
def binom_rec(n, k):
    if n < 0 or k < 0 or k > n:
        return 0
    elif n >= 0 and k == 0:
        return 1
    else:
        return binom_rec(n - 1, k - 1) + binom_rec(n - 1, k)

In [21]:
binom_rec(12, 4)

495

````{admonition} Question
 Écrire une fonction `binom_dp(n, k)` renvoyant $\binom{n}{k}$ en utilisant la même formule, mais par programmation dynamique.  
Pour cela, on pourra stocker $\binom{n}{k}$ dans une matrice (ou : un dictionnaire) et la remplir par $n$ croissant et par $k$ croissant.
````

In [22]:
def binom_dp(n, k):
    M = [[0 for _ in range(k + 1)] for _ in range(n + 1)]
    # M[i][j] contiendra j parmi i
    for i in range(0, n + 1):
        M[i][0] = 1 # cas de base
        for j in range(1, k + 1):
            M[i][j] = M[i - 1][j- 1] + M[i - 1][j] # récurrence
    return M[n][k]

In [23]:
binom_dp(56, 24)

4355031703297275

````{admonition} Question
 Écrire une fonction `binom_memo(n, k)` renvoyant $\binom{n}{k}$ en utilisant le même principe, mais avec mémoïsation plutôt que programmation dynamique.
````

In [24]:
def binom_memo(n, k):
    d = {} # d[(i, j)] contiendra j parmi i
    def aux(i, j): # renvoie j parmi i
        if i < 0 or j < 0 or j > i : # cas de base
            d[(i, j)] = 0
            return 0
        elif i >= 0 and j == 0:
            d[(i, j)] = 1
            return 1
        else: # dans le cas général, regarder si (i, j) est dans d : si oui, renvoyer la valeur associée, sinon la calculer et l'ajouter à d
            if (i, j) in d:
                return d[(i , j)]
            else:
                coef = aux(i - 1, j - 1) + aux(i - 1, j)
                d[(i , j)] = coef
                return coef
    return aux(n, k)

In [25]:
binom_memo(56, 24)

4355031703297275

## Rendu de monnaie

Étant donnée une liste `L` d'entiers $a_1,\ldots,a_k$ (des pièces), on veut calculer le nombre minimum $r(n, k)$ de pièces parmi $a_1, ..., a_k$ dont la somme vaut $n$.

Par exemple, si $k = 3$ et $a_1 = 1, a_2 = 2, a_3 = 5$ alors $r(7, 3) = 2$ (car $7 = 2 + 5$ et c'est la façon de rendre $7$€ qui utilise le moins de pièces).  

Remarques :  
- On peut utiliser plusieurs fois la même pièce.  
- $r(0, k)$ revient à rendre $0$€, ce qu'on peut faire avec $0$ pièce : $r(0, k) = 0$
- $r(n, 0)$ revient à n'utiliser aucune pièce, ce qui est impossible si $n \neq 0$ : on posera $r(n, 0)$ = $\infty$ (`float("inf")` en Python).

````{admonition} Question
 Écrire une relation de récurrence sur $r(n, k)$. On pourra distinguer deux cas pour rendre $n$ euros avec les picèes $a_1$, ..., $a_k$ :  
- soit $a_k$ n'est pas utilisée (et on a donc $r(n, k) = r(n, k - 1)$)  
- soit $a_k$ est utilisée (et on a $r(n, k) = ...$).
````

````{admonition} Question
 En déduire une fonction `rendu(L, n)` par programmation dynamique renvoyant le nombre minimum de pièces requises pour rendre `n` euros, où `L` est la liste des pièces.  
On remplira une matrice `R` pour que `R[i][j]` contienne le nombre minimum de pièces pour rendre `i` euros en utilisant les `j` premières pièces de `L`.
````

In [26]:
def rendu(L, n):
    R = [[0 for _ in range(len(L) + 1)] for _ in range(n + 1)]
    for somme in range(n + 1):
        for piece in range(len(L) + 1):
            if somme == 0:
                R[somme][piece] = 0
            elif piece == 0:
                R[somme][piece] = float("inf")
            else:
                if L[piece - 1] > somme:
                    R[somme][piece] = R[somme][piece - 1]
                else:
                    R[somme][piece] = min(1 + R[somme - L[piece - 1]][piece], R[somme][piece - 1])
    return R[n][len(L)]

In [27]:
rendu([1, 2, 4, 5], 8)

2

````{admonition} Question
 Réécrire la fonction précédente par mémoïsation plutôt que par programmation dynamique.
````

In [28]:
def rendu_mem(L, n):
    rendu = {}
    def aux(somme, piece):
        if somme == 0:
            rendu[(somme, piece)] = 0
        elif piece == 0:
            rendu[(somme, piece)] = float("inf")
        else:
            if (somme, piece)  not in rendu:
                if L[piece  - 1] > somme:
                    rendu[(somme, piece)] = aux(somme, piece - 1)
                else:
                    rendu[(somme, piece)] = min(1 + aux(somme - L[piece - 1], piece), aux(somme, piece - 1))
        return rendu[(somme, piece)]
    return aux(n, len(L))


In [29]:
rendu_mem([1, 2, 5], 8)

3

## Plus grand carré dans une matrice

Étant donnée une matrice carrée remplie de 0 ou 1, on souhaite connaître la taille du plus gros carré de 1 dans cette matrice.  
Par exemple, ce nombre est 2 pour la matrice $M$ suivante (correspondant au carré en pointillé) :

<center><img src=https://raw.githubusercontent.com/fortierq/tikz/master/dyn_prog/matrix_square/matrix_square.png width=200></center>


La case de coordonnés $(x, y)$ est celle sur la ligne $x$, colonne $y$. La case de coordonnées (0, 0) est celle en haut à gauche.  
On supposera que les indices en arguments des fonctions ne dépassent pas des tableaux ou matrices correspondants.

````{admonition} Question
 Définir `M` en Python.
````

In [57]:
M = [[0, 1, 0, 0],
     [0, 0, 1, 1],
     [0, 1, 1, 1],
     [1, 1, 1, 0]]

M_prim = [
    [0, 1, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
]

## Méthode naïve

````{admonition} Question
 Écrire une fonction `est_carre` telle que `est_carre(m, x, y, k)` détermine si la sous-matrice de `m` de taille $k \times k$ et dont la case en haut à gauche a pour coordonnées (`x`, `y`) ne possède que des 1.
````

In [58]:
def est_carre(m, x, y, k):
    flag = True
    for i in range(x, x + k):
        for j in range(y, y + k):
            flag *= m[i][j]
    return bool(flag)

est_carre(M_prim, 1, 2, 2)

False

In [59]:
assert(est_carre(M, 1, 2, 2) and not est_carre(M, 1, 1, 2))

````{admonition} Question
 Écrire une fonction `contient_carre` telle que `contient_carre(m, k)` renvoie `true` si `m` contient un carré de 1 de taille $k$, `false` sinon.
````

In [60]:
def contient_carre(m, k):
        if k > len(m):
            return False
        else:
            for x in range(len(m) + 1 - k):
                for y in range(len(m) + 1 - k):
                    if est_carre(m, x, y, k):
                        return True
            return False

In [61]:
assert(contient_carre(M, 2) and not contient_carre(M, 3))

````{admonition} Question
 Écrire une fonction `max_carre1` telle que `max_carre1(m)` renvoie la taille maximum d'un carré de 1 contenu dans `m`.
````

In [62]:
def max_carre1(m):
    k = 0
    while contient_carre(m, k + 1):
        k += 1
    return k

In [63]:
max_carre1(M_prim)

1

````{admonition} Question
 Quelle est la complexité de `max_carre1(m)` dans le pire cas ?
````

On va construire une matrice `c` telle que `c[x][y]` est la taille maximum d'un carré de 1 dans `m` dont la case en bas à droite est `m[x][y]` (c'est à dire un carré de 1 qui contient `m[x][y]` mais aucun `m[i][j]` si $i > x$ ou $j > y$).  
Par exemple, `c[1][2] = 1` et `c[2][3] = 2` pour la matrice $M$ ci-dessus.

````{admonition} Question
 Que vaut `c[0][y]` et `c[x][0]` ?
````

On a toujours `c[0][y] = m[0][y]` et `c[x][0] = m[x][0]`

````{admonition} Question
 Que vaut `c[x][y]` si `m[x][y] = 0` ?
````

Dans ce cas-ci : `c[x][y] = 0`

````{admonition} Question
 Montrer que, si `m[x][y] = 1`, `c[x][y] = 1 + min(c[x-1][y], c[x][y-1], c[x-1][y-1])`.
````

CA MARCHE

````{admonition} Question
 En déduire une fonction `max_carre2` telle que `max_carre2(m)` renvoie la taille maximum d'un carré de 1 contenu dans `m`, ainsi que les coordonnées de la case en haut à gauche d'un tel carré.
````

In [65]:
def max_carre2(m):
    c = [[0]*len(m) for _ in range(len(m))]
    taille_max, coordX, coordY = 0, 0, 0
    for x in range(len(m)):
        for y in range(len(m)):
            if x == 0 or y==0:
                c[x][y] = m[x][y]
            elif m[x][y] == 0:
                c[x][y] = 0
            else:
                c[x][y] = 1 + min(c[x - 1][y], c[x][y - 1], c[x - 1][y - 1])
                taille_max, coordX, coordY = max((taille_max, coordX, coordY), (c[x][y], x - c[x][y] + 1, y - c[x][y] + 1))
    return taille_max, coordX, coordY

In [66]:
max_carre2(M)

(2, 2, 1)

````{admonition} Question
 Quelle est la complexité de `max_carre2(m)`, en fonction des dimensions de `m`? Comparer avec `max_carre1(m)`.
````

## Pour ceux qui ont fini

Cette partie n'est pas à rendre, sauf si vous en avez le temps.

````{admonition} Question
 S'inscrire sur [https://projecteuler.net/](https://projecteuler.net/) et résoudre [ce problème](https://projecteuler.net/problem=67).  
On pourra télécharger le fichier triangle.txt demandé avec :  
```python
import urllib.request
f = urllib.request.urlopen("https://projecteuler.net/project/resources/p067_triangle.txt")
lignes = list(map(lambda x : list(map(int, x.split())), f.readlines())) # renvoie la liste des lignes du fichier
```
````

````{admonition} Question
 [Résoudre ce problème (en s'inscrivant préalablement)](https://leetcode.com/problems/longest-increasing-subsequence)
````